# Factor risk models

In [1]:
import pandas as pd
import numpy as np
import cvxpy as cvx

from cvx.linalg import pca
from cvx.risk.factor import FactorModel

## Load prices and compute returns

In [ ]:
prices = pd.read_csv("data/stock_prices.csv", index_col=0, header=0, parse_dates=True)
returns = prices.pct_change().fillna(0.0)

## Compute principal components

In [ ]:
factors = pca(returns=returns, n_components=10)

## Create the risk model, here a FactorModel

In [ ]:
model = FactorModel(assets=len(returns.columns), k=10)

# update the model parameters
model.update(cov=factors.cov, exposure=factors.exposure.values, idiosyncratic_risk=factors.idiosyncratic.std().values)

# test the risk model with uniform weights
weights = 0.05 * np.ones(20)
model.estimate(weights).value

## RiskModel is injected into optimizer

In [ ]:
w = cvx.Variable(20)
y = cvx.Variable(10)

objective = cvx.Minimize(1e3*model.estimate(w, y=y))
constraints = [w >= 0, cvx.sum(w) == 1] + model.bounds.constraints(w)

problem = cvx.Problem(objective=objective, constraints=constraints)
problem.solve()

print(pd.Series(data=w.value, index=prices.columns))
print(model.estimate(w, y=y).value)

# check the solution
assert np.isclose(w.value.sum(), 1.0)
assert np.all(w.value > -0.01)